In [1]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.multiprocessing as mp

import torch
import torch.nn as nn
import torch.utils.data as utils
from sklearn.metrics import accuracy_score

# import my lib
from models.cnn import CNN as net
from lib.federated import FederatedServer

mp.set_start_method('spawn')

In [2]:
DEVICE_LIST = ['cuda:' + str(i) for i in range(4)]
ROUNDS = 10

WARM_SETTINGS = {
    'warm_up': False,
    'setting':{
        'batch_size': 128
    }
}

NET_KWARGS = {
    'neurons': 128
}

CLIENT_SETTINGS = {
    'mode': 'thres',
    'thres': 0.99,
    'batch_size': 128,
    'lr': 1e-4,
    'enable_scheduler': True,
    'eval_each_iter': False,
    'loss_func': nn.CrossEntropyLoss,
    'optimizer': optim.Adam,
    'scheduler': optim.lr_scheduler.ReduceLROnPlateau,
    'scheduler_settings':{
        'mode': 'min',
        'factor': 0.13,
        'patience': 10,
        'verbose': True,
        'min_lr': 1e-8,
        'cooldown': 3
    }
}

SERVER_SETTINGS = {
    'split_method': 'imba-size',
    'capacity': [10**i/11111 for i in range(5)],
    'imbalanced_rate': 0.99,
    'clients_num': 5,
    'random_response': False,
    'client_settings': CLIENT_SETTINGS,
    'warm_setting': WARM_SETTINGS,
    'net_kwargs': NET_KWARGS,
    'cal_sv': True,
    'cal_loo': True,
    'eval_clients': True,
    'devices': DEVICE_LIST
}

TRANSFORM_MNIST = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.13066062, ), (0.30810776, ))
])

trainset = torchvision.datasets.MNIST('../public_set', train=True, transform=TRANSFORM_MNIST, download=True)
testset = torchvision.datasets.MNIST('../public_set', train=False, transform=TRANSFORM_MNIST, download=False)


In [3]:
# run
fl = FederatedServer(net, trainset, testset, **SERVER_SETTINGS)
fl.run(rounds=ROUNDS)
fl.save_valuation('differ_quan_')

Start training clients...
Start evaluating parameters from clients...
Start to calculate shapley values...
Client 0's SV is: 0.012909174311926605
Client 1's SV is: 0.15407614678899087
Client 2's SV is: 0.2687284403669725
Client 3's SV is: 0.23743302752293569
Client 4's SV is: 0.3138532110091743
Start calculating LOO values...
0 worker's LOO value: 0.000000
1 worker's LOO value: 0.000000
3 worker's LOO value: -0.000600
2 worker's LOO value: 0.000000
4 worker's LOO value: 0.014900
Rounds[1/10]: Loss: 0.04133459092470045 | Test Accu: 0.987 | Time Elapse: 940.4570653438568
--------------------
Start evaluating parameters from clients...
Start to calculate shapley values...
Client 0's SV is: 0.19924220183486235
Client 1's SV is: 0.18127614678899084
Client 2's SV is: 0.20822385321100917
Client 3's SV is: 0.24416788990825689
Client 4's SV is: 0.1548899082568807
Start calculating LOO values...
1 worker's LOO value: 0.000000
0 worker's LOO value: 0.000000
2 worker's LOO value: -0.000100
3 worke

In [4]:
fl.result

defaultdict(dict,
            {0: {1: {'iter': 20,
               'eval': 0.1001,
               'SV': 0.012909174311926605,
               'LOO': 0.0},
              2: {'iter': 1,
               'eval': 0.9872,
               'SV': 0.19924220183486235,
               'LOO': 0.0},
              3: {'iter': 1,
               'eval': 0.9871,
               'SV': 0.19923119266055048,
               'LOO': 0.0},
              4: {'iter': 1,
               'eval': 0.9869,
               'SV': 0.15392018348623854,
               'LOO': 0.0},
              5: {'iter': 1,
               'eval': 0.9881,
               'SV': 0.22664311926605504,
               'LOO': 0.0},
              6: {'iter': 1,
               'eval': 0.9873,
               'SV': 0.17209357798165137,
               'LOO': 0.0},
              7: {'iter': 1,
               'eval': 0.9881,
               'SV': 0.20850000000000002,
               'LOO': 0.0},
              8: {'iter': 1,
               'eval': 0.9888,
       